# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [2]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [3]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [4]:
# Your code here
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)



In [5]:
# Your code here
lin = LinearRegression()
# Create a LinearRegression model and fit it on scaled training data
lin.fit(X_train, y_train)
# Calculate a baseline r-squared score on training data
baseline_train = lin.score(X_train, y_train)
baseline_val = lin.score(X_val, y_val)

baseline_train, baseline_val

(0.7868344817421309, 0.6375622643038104)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [7]:
# Your code here

interactions = []

# Find combinations of columns and loop over them
column_pairs = list(combinations(X_train.columns, 2))
for (col1, col2) in column_pairs:
    # Make copies of X_train and X_val
    features_train = X_train.copy()
    features_val = X_val.copy()
    
    # Add interaction term to data
    features_train["interaction"] = features_train[col1] * features_train[col2]
    features_val["interaction"] = features_val[col1] * features_val[col2]
    
    # Find r-squared score (fit on training data, evaluate on test data)
    score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
    
    # Append to data structure
    interactions.append(((col1, col2), score))

# Sort and subset the data structure to find the top 7
top_7_interactions = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
print("Top 7 interactions:")
print(top_7_interactions)




Top 7 interactions:
[(('LotArea', '1stFlrSF'), 0.7211105666140567), (('LotArea', 'TotalBsmtSF'), 0.7071649207050106), (('LotArea', 'GrLivArea'), 0.6690980823779026), (('LotArea', 'Fireplaces'), 0.6529699515652588), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.647299489040519), (('OverallCond', 'TotalBsmtSF'), 0.642901987923377), (('OverallQual', '2ndFlrSF'), 0.6422324294284367)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [8]:
# Your code here

# Loop over top 7 interactions
for record in top_7_interactions:
    # Extract column names from data structure
    col1, col2 = record[0]
    
    # Construct new column name
    new_col_name = col1 + "_" + col2
    
    # Add new column to X_train and X_val
    X_train[new_col_name] = X_train[col1] * X_train[col2]
    X_val[new_col_name] = X_val[col1] * X_val[col2]
    
X_train


,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023,0.477622,0.271823,0.720306,-0.313023,1.789339,0.260039,0.080686
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820,-0.097522,-0.158985,0.148966,0.307643,0.295392,-0.261809,0.080686
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023,-0.107471,-0.061263,0.095069,0.070221,1.460730,0.261252,1.064983
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820,0.003613,0.002418,0.000789,0.002704,-0.286483,-1.160508,-0.078252


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [12]:
polynomials = []

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2, 3, 4):
        
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(features_train[[col]]))
        col_transformed_val = pd.DataFrame(poly.transform(features_val[[col]]))
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train], axis=1)
        features_val = pd.concat([features_val.drop(col, axis=1), col_transformed_val], axis=1)
    
        # Find r-squared score
        score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
    
        # Append to data structure
        polynomials.append((col, degree, score))
    
# Sort and subset the data structure to find the top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1], reverse=True)[:7]
print("Top 7 polynomials:")
print(top_7_polynomials)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [11]:
# Your code here
top_polynomials = pd.DataFrame(top_7_polynomials, columns=["Column", "Degree", "R^2"])
top_polynomials

# Drop duplicate columns based on Column name
top_polynomials.drop_duplicates(subset="Column", inplace=True)
top_polynomials

# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    #Create polynomimal terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                              columns=poly.get_feature_names([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]),
                              columns=poly.get_feature_names([col]))
    
    #Concat back to original
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)

    

NameError: name 'top_7_polynomials' is not defined

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [13]:
# Your code here
X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [14]:
# Your code here

lr = LinearRegression()
lr.fit(X_train, y_train)

print("Train R^2:", lr.score(X_train, y_train))
print("Validation R^2: ", lr.score(X_val, y_val))

Train R^2: 0.8003632679622383
Validation R^2:  0.7468205440300999


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [15]:
# Your code here

for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_val, y_val))
    print(f"Using {n} out of {X_train.shape[1]} features")
    print()


Train R^2: 0.776039994126505
Test R^2:  0.6352981725272362
Using 5 out of 17 features

Train R^2: 0.7956644353038372
Test R^2:  0.724748019762746
Using 10 out of 17 features

Train R^2: 0.8003459743341153
Test R^2:  0.7467783601803735
Using 15 out of 17 features

Train R^2: 0.8003632679622383
Test R^2:  0.7468205440300999
Using 20 out of 17 features

Train R^2: 0.8003632679622383
Test R^2:  0.7468205440300999
Using 25 out of 17 features



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [16]:
# Your code here

for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Train R^2: 0.8003631205043212
Validation R^2:  0.7467595180829177
Using 26 out of 17 features
and an alpha of 1

Train R^2: 0.8003615882305483
Validation R^2:  0.7464817233440981
Using 26 out of 17 features
and an alpha of 10

Train R^2: 0.8002166242199933
Validation R^2:  0.7434128637149325
Using 26 out of 17 features
and an alpha of 100

Train R^2: 0.7958866297543004
Validation R^2:  0.6912680517975045
Using 21 out of 17 features
and an alpha of 1000

Train R^2: 0.7503810493218819
Validation R^2:  0.6485821732264372
Using 16 out of 17 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [17]:
# Your written answer here
"""
For RFE the model with the best validation R-Squared was using 20 features

For Lasso the model with the best validation R-Squared was using an alpha of 10000

The Lasso result was a bit better so let's go with that and the 12 features
that it selected
"""

"\nFor RFE the model with the best validation R-Squared was using 20 features\n\nFor Lasso the model with the best validation R-Squared was using an alpha of 10000\n\nThe Lasso result was a bit better so let's go with that and the 12 features\nthat it selected\n"

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [18]:
# Your code here

X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add interactions to X_test
for record in top_7_interactions:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

# Add polynomials to X_val
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

NameError: name 'top_polynomials' is not defined

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [19]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

R-Squared: 0.7524647427519646
MSE: 1734060978.3728046


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.